In [1]:
#pip install opencv-python
#sudo pip install pandas

In [2]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 
# Importing the libraries
import matplotlib.pyplot as plt
#import warnings
#warnings.filterwarnings('ignore')

import cv2
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from skimage.transform import resize

In [3]:
data = pd.read_csv('aoi/aoi/train.csv',engine='python')
data.tail() 

,ID,Label
2523,train_02523.png,0
2524,train_02524.png,0
2525,train_02525.png,3
2526,train_02526.png,0
2527,train_02527.png,5


In [4]:
data

,ID,Label
0,train_00000.png,0
1,train_00001.png,1
2,train_00002.png,1
3,train_00003.png,5
4,train_00004.png,5
...,...,...
2523,train_02523.png,0
2524,train_02524.png,0
2525,train_02525.png,3
2526,train_02526.png,0


In [5]:
data.shape

(2528, 2)

In [6]:
data_lebel=data.iloc[:,1].values
data_lebel = data_lebel.reshape(-1,1)
data_lebel = data_lebel.astype("float32")
data_lebel.shape

(2528, 1)

In [7]:
data_lebel[:5]

array([[0.],
       [1.],
       [1.],
       [5.],
       [5.]], dtype=float32)

In [8]:
train_dir = 'aoi/aoi/train_images'
nTrain = len(os.listdir("aoi/aoi/train_images/train_images"))
print(nTrain)

2528


In [9]:
def preprocess_fn(image_path):
    '''
    Image pre-processing.
    Opens image as grayscale, adds channel dimension, normalizes to range 0:1
    and then scales by input quantization scaling factor
    input arg: path of image file
    return: numpy array
    '''
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image=resize(image,(224,224,3))
    #print(image.shape)
    image = image * (1/255.0)
    image = image.astype(np.float32)
    return image

In [10]:
img=[]
for i in range(nTrain):
    path=os.path.join('aoi/aoi/train_images/train_images',os.listdir("aoi/aoi/train_images/train_images")[i])
    img.append(preprocess_fn(path))

In [11]:
img

[array([[[0.00238199, 0.00238199, 0.00238199],
         [0.0025039 , 0.0025039 , 0.0025039 ],
         [0.00261846, 0.00261846, 0.00261846],
         ...,
         [0.00283242, 0.00283242, 0.00283242],
         [0.0027069 , 0.0027069 , 0.0027069 ],
         [0.00271167, 0.00271167, 0.00271167]],
 
        [[0.00260776, 0.00260776, 0.00260776],
         [0.0027227 , 0.0027227 , 0.0027227 ],
         [0.00262763, 0.00262763, 0.00262763],
         ...,
         [0.00267052, 0.00267052, 0.00267052],
         [0.00261534, 0.00261534, 0.00261534],
         [0.00278423, 0.00278423, 0.00278423]],
 
        [[0.00316461, 0.00316461, 0.00316461],
         [0.00297175, 0.00297175, 0.00297175],
         [0.00273217, 0.00273217, 0.00273217],
         ...,
         [0.00285158, 0.00285158, 0.00285158],
         [0.0027983 , 0.0027983 , 0.0027983 ],
         [0.00286621, 0.00286621, 0.00286621]],
 
        ...,
 
        [[0.00256275, 0.00256275, 0.00256275],
         [0.00277671, 0.00277671, 0.00277

In [12]:
image=np.array(img)

In [13]:
image.shape

(2528, 224, 224, 3)

In [14]:
image[0]

array([[[0.00238199, 0.00238199, 0.00238199],
        [0.0025039 , 0.0025039 , 0.0025039 ],
        [0.00261846, 0.00261846, 0.00261846],
        ...,
        [0.00283242, 0.00283242, 0.00283242],
        [0.0027069 , 0.0027069 , 0.0027069 ],
        [0.00271167, 0.00271167, 0.00271167]],

       [[0.00260776, 0.00260776, 0.00260776],
        [0.0027227 , 0.0027227 , 0.0027227 ],
        [0.00262763, 0.00262763, 0.00262763],
        ...,
        [0.00267052, 0.00267052, 0.00267052],
        [0.00261534, 0.00261534, 0.00261534],
        [0.00278423, 0.00278423, 0.00278423]],

       [[0.00316461, 0.00316461, 0.00316461],
        [0.00297175, 0.00297175, 0.00297175],
        [0.00273217, 0.00273217, 0.00273217],
        ...,
        [0.00285158, 0.00285158, 0.00285158],
        [0.0027983 , 0.0027983 , 0.0027983 ],
        [0.00286621, 0.00286621, 0.00286621]],

       ...,

       [[0.00256275, 0.00256275, 0.00256275],
        [0.00277671, 0.00277671, 0.00277671],
        [0.00239242, 0

In [15]:
x_train=image

In [16]:
y_train=data_lebel

In [17]:
print(x_train.shape)
print(y_train.shape)

(2528, 224, 224, 3)
(2528, 1)


In [18]:
y_train_OneHot = keras.utils.to_categorical(y_train)

In [19]:
print(y_train_OneHot)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [20]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
ResNet152V2_conv = ResNet152V2(weights='imagenet',
                  include_top=False)
ResNet152V2_conv.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                     

                                                                                                  
 conv2_block2_out (Add)         (None, None, None,   0           ['conv2_block1_out[0][0]',       
                                256)                              'conv2_block2_3_conv[0][0]']    
                                                                                                  
 conv2_block3_preact_bn (BatchN  (None, None, None,   1024       ['conv2_block2_out[0][0]']       
 ormalization)                  256)                                                              
                                                                                                  
 conv2_block3_preact_relu (Acti  (None, None, None,   0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                        256)                                                              
                                                                                                  
 conv2_blo

                                128)                             ]                                
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_1_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, None, None,   0          ['conv3_block2_1_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block2_2_pad (ZeroPaddin  (None, None, None,   0          ['conv3_block2_1_relu[0][0]']    
 g2D)                           128)                                                              
          

 conv3_block4_2_relu (Activatio  (None, None, None,   0          ['conv3_block4_2_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block4_3_conv (Conv2D)   (None, None, None,   66048       ['conv3_block4_2_relu[0][0]']    
                                512)                                                              
                                                                                                  
 conv3_block4_out (Add)         (None, None, None,   0           ['conv3_block3_out[0][0]',       
                                512)                              'conv3_block4_3_conv[0][0]']    
                                                                                                  
 conv3_block5_preact_bn (BatchN  (None, None, None,   2048       ['conv3_block4_out[0][0]']       
 ormalizat

                                                                                                  
 conv3_block7_1_bn (BatchNormal  (None, None, None,   512        ['conv3_block7_1_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block7_1_relu (Activatio  (None, None, None,   0          ['conv3_block7_1_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block7_2_pad (ZeroPaddin  (None, None, None,   0          ['conv3_block7_1_relu[0][0]']    
 g2D)                           128)                                                              
                                                                                                  
 conv3_blo

...

                                                                                                  
 conv4_block29_1_bn (BatchNorma  (None, None, None,   1024       ['conv4_block29_1_conv[0][0]']   
 lization)                      256)                                                              
                                                                                                  
 conv4_block29_1_relu (Activati  (None, None, None,   0          ['conv4_block29_1_bn[0][0]']     
 on)                            256)                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, None, None,   0          ['conv4_block29_1_relu[0][0]']   
 ng2D)                          256)                                                              
                                                                                                  
 conv4_blo

 on)                            256)                                                              
                                                                                                  
 conv4_block31_3_conv (Conv2D)  (None, None, None,   263168      ['conv4_block31_2_relu[0][0]']   
                                1024)                                                             
                                                                                                  
 conv4_block31_out (Add)        (None, None, None,   0           ['conv4_block30_out[0][0]',      
                                1024)                             'conv4_block31_3_conv[0][0]']   
                                                                                                  
 conv4_block32_preact_bn (Batch  (None, None, None,   4096       ['conv4_block31_out[0][0]']      
 Normalization)                 1024)                                                             
          

 conv4_block34_1_bn (BatchNorma  (None, None, None,   1024       ['conv4_block34_1_conv[0][0]']   
 lization)                      256)                                                              
                                                                                                  
 conv4_block34_1_relu (Activati  (None, None, None,   0          ['conv4_block34_1_bn[0][0]']     
 on)                            256)                                                              
                                                                                                  
 conv4_block34_2_pad (ZeroPaddi  (None, None, None,   0          ['conv4_block34_1_relu[0][0]']   
 ng2D)                          256)                                                              
                                                                                                  
 conv4_block34_2_conv (Conv2D)  (None, None, None,   589824      ['conv4_block34_2_pad[0][0]']    
          

                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, None, None,   0          ['conv4_block35_out[0][0]']      
                                1024)                                                             
                                                                                                  
 conv4_block36_3_conv (Conv2D)  (None, None, None,   263168      ['conv4_block36_2_relu[0][0]']   
                                1024)                                                             
                                                                                                  
 conv4_block36_out (Add)        (None, None, None,   0           ['max_pooling2d_2[0][0]',        
                                1024)                             'conv4_block36_3_conv[0][0]']   
                                                                                                  
 conv5_blo

 vation)                        2048)                                                             
                                                                                                  
 conv5_block3_1_conv (Conv2D)   (None, None, None,   1048576     ['conv5_block3_preact_relu[0][0]'
                                512)                             ]                                
                                                                                                  
 conv5_block3_1_bn (BatchNormal  (None, None, None,   2048       ['conv5_block3_1_conv[0][0]']    
 ization)                       512)                                                              
                                                                                                  
 conv5_block3_1_relu (Activatio  (None, None, None,   0          ['conv5_block3_1_bn[0][0]']      
 n)                             512)                                                              
          

In [21]:
features=ResNet152V2_conv.predict(x_train)

79/79 [==============================] - 325s 4s/step


In [22]:
features.shape

(2528, 7, 7, 2048)

In [23]:
train_features = np.reshape(features, (2528, 7*7*2048))

In [24]:
from keras import models
from keras import layers
from keras import optimizers
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=7*7*2048))
model.add(layers.Dense(512, activation='relu', input_dim=7*7*2048))
model.add(layers.Dense(6, activation='softmax'))

In [25]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=2e-4), metrics=['acc'])
train_history=model.fit(train_features, y_train_OneHot,
                        validation_split=0.1,
                        epochs=500, batch_size=20, verbose=1)   

C:\Users\lu524\anaconda3\lib\site-packages\keras\optimizers\legacy\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/500
114/114 [==============================] - 140s 1s/step - loss: 1.7168 - acc: 0.4084 - val_loss: 1.4255 - val_acc: 0.4190
Epoch 2/500
114/114 [==============================] - 133s 1s/step - loss: 1.1675 - acc: 0.5371 - val_loss: 0.9928 - val_acc: 0.6166
Epoch 3/500
114/114 [==============================] - 132s 1s/step - loss: 1.0383 - acc: 0.5785 - val_loss: 0.8094 - val_acc: 0.6917
Epoch 4/500
114/114 [==============================] - 132s 1s/step - loss: 0.9593 - acc: 0.6163 - val_loss: 0.8691 - val_acc: 0.6838
Epoch 5/500
114/114 [==============================] - 133s 1s/step - loss: 0.9070 - acc: 0.6400 - val_loss: 0.8883 - val_acc: 0.5771
Epoch 6/500
114/114 [==============================] - 132s 1s/step - loss: 0.8847 - acc: 0.6347 - val_loss: 0.8558 - val_acc: 0.5771
Epoch 7/500
114/114 [==============================] - 131s 1s/step - loss: 0.8488 - acc: 0.6334 - val_loss: 0.7136 - val_acc: 0.6719
Epoch 8/500
114/114 [==============================] - 132s 1s

Epoch 62/500
114/114 [==============================] - 132s 1s/step - loss: 0.4236 - acc: 0.8321 - val_loss: 0.5561 - val_acc: 0.7708
Epoch 63/500
114/114 [==============================] - 130s 1s/step - loss: 0.4071 - acc: 0.8396 - val_loss: 0.4690 - val_acc: 0.8300
Epoch 64/500
114/114 [==============================] - 132s 1s/step - loss: 0.3875 - acc: 0.8462 - val_loss: 0.4349 - val_acc: 0.7905
Epoch 65/500
114/114 [==============================] - 131s 1s/step - loss: 0.4040 - acc: 0.8352 - val_loss: 0.5043 - val_acc: 0.8261
Epoch 66/500
114/114 [==============================] - 131s 1s/step - loss: 0.4027 - acc: 0.8422 - val_loss: 0.3432 - val_acc: 0.8735
Epoch 67/500
114/114 [==============================] - 132s 1s/step - loss: 0.3978 - acc: 0.8488 - val_loss: 0.4333 - val_acc: 0.8379
Epoch 68/500
114/114 [==============================] - 132s 1s/step - loss: 0.3821 - acc: 0.8484 - val_loss: 0.3707 - val_acc: 0.8696
Epoch 69/500
114/114 [==============================] -

114/114 [==============================] - 156s 1s/step - loss: 0.2974 - acc: 0.8791 - val_loss: 0.4067 - val_acc: 0.8379
Epoch 123/500
114/114 [==============================] - 159s 1s/step - loss: 0.3007 - acc: 0.8875 - val_loss: 0.3212 - val_acc: 0.8735
Epoch 124/500
114/114 [==============================] - 156s 1s/step - loss: 0.2905 - acc: 0.8879 - val_loss: 0.7842 - val_acc: 0.7628
Epoch 125/500
114/114 [==============================] - 156s 1s/step - loss: 0.2999 - acc: 0.8853 - val_loss: 0.4147 - val_acc: 0.8221
Epoch 126/500
114/114 [==============================] - 157s 1s/step - loss: 0.2952 - acc: 0.8853 - val_loss: 0.3824 - val_acc: 0.8300
Epoch 127/500
114/114 [==============================] - 156s 1s/step - loss: 0.2992 - acc: 0.8910 - val_loss: 0.2877 - val_acc: 0.8893
Epoch 128/500
114/114 [==============================] - 156s 1s/step - loss: 0.2902 - acc: 0.8848 - val_loss: 0.3103 - val_acc: 0.9091
Epoch 129/500
114/114 [==============================] - 155s 

114/114 [==============================] - 131s 1s/step - loss: 0.2610 - acc: 0.9015 - val_loss: 0.4251 - val_acc: 0.8656
Epoch 183/500
114/114 [==============================] - 132s 1s/step - loss: 0.2501 - acc: 0.9099 - val_loss: 0.7266 - val_acc: 0.7826
Epoch 184/500
114/114 [==============================] - 131s 1s/step - loss: 0.2462 - acc: 0.9015 - val_loss: 0.4026 - val_acc: 0.8656
Epoch 185/500
114/114 [==============================] - 131s 1s/step - loss: 0.2399 - acc: 0.9042 - val_loss: 0.4105 - val_acc: 0.9091
Epoch 186/500
114/114 [==============================] - 132s 1s/step - loss: 0.2684 - acc: 0.9011 - val_loss: 0.4375 - val_acc: 0.8458
Epoch 187/500
114/114 [==============================] - 131s 1s/step - loss: 0.2297 - acc: 0.9125 - val_loss: 0.4762 - val_acc: 0.7747
Epoch 188/500
114/114 [==============================] - 132s 1s/step - loss: 0.2307 - acc: 0.9099 - val_loss: 0.5100 - val_acc: 0.8617
Epoch 189/500
114/114 [==============================] - 131s 

114/114 [==============================] - 132s 1s/step - loss: 0.2263 - acc: 0.9178 - val_loss: 0.4454 - val_acc: 0.8498
Epoch 243/500
114/114 [==============================] - 133s 1s/step - loss: 0.2210 - acc: 0.9191 - val_loss: 0.5275 - val_acc: 0.8696
Epoch 244/500
114/114 [==============================] - 133s 1s/step - loss: 0.2324 - acc: 0.9134 - val_loss: 0.3429 - val_acc: 0.9012
Epoch 245/500
114/114 [==============================] - 132s 1s/step - loss: 0.2313 - acc: 0.9134 - val_loss: 0.5032 - val_acc: 0.8814
Epoch 246/500
114/114 [==============================] - 133s 1s/step - loss: 0.2094 - acc: 0.9231 - val_loss: 0.6228 - val_acc: 0.7905
Epoch 247/500
114/114 [==============================] - 131s 1s/step - loss: 0.2116 - acc: 0.9248 - val_loss: 0.5354 - val_acc: 0.8696
Epoch 248/500
114/114 [==============================] - 133s 1s/step - loss: 0.2383 - acc: 0.9130 - val_loss: 0.3385 - val_acc: 0.8814
Epoch 249/500
114/114 [==============================] - 134s 

114/114 [==============================] - 132s 1s/step - loss: 0.2305 - acc: 0.9169 - val_loss: 0.6136 - val_acc: 0.8538
Epoch 303/500
114/114 [==============================] - 131s 1s/step - loss: 0.2050 - acc: 0.9262 - val_loss: 0.4013 - val_acc: 0.9012
Epoch 304/500
114/114 [==============================] - 130s 1s/step - loss: 0.2141 - acc: 0.9262 - val_loss: 0.3351 - val_acc: 0.9170
Epoch 305/500
114/114 [==============================] - 131s 1s/step - loss: 0.2225 - acc: 0.9231 - val_loss: 0.2520 - val_acc: 0.9368
Epoch 306/500
114/114 [==============================] - 131s 1s/step - loss: 0.1898 - acc: 0.9345 - val_loss: 1.1617 - val_acc: 0.8024
Epoch 307/500
114/114 [==============================] - 133s 1s/step - loss: 0.2248 - acc: 0.9222 - val_loss: 0.3330 - val_acc: 0.9130
Epoch 308/500
114/114 [==============================] - 132s 1s/step - loss: 0.2157 - acc: 0.9235 - val_loss: 0.3756 - val_acc: 0.8696
Epoch 309/500
114/114 [==============================] - 132s 

114/114 [==============================] - 157s 1s/step - loss: 0.1918 - acc: 0.9323 - val_loss: 0.3891 - val_acc: 0.8696
Epoch 363/500
114/114 [==============================] - 156s 1s/step - loss: 0.1843 - acc: 0.9327 - val_loss: 0.4109 - val_acc: 0.8933
Epoch 364/500
114/114 [==============================] - 156s 1s/step - loss: 0.2247 - acc: 0.9257 - val_loss: 0.7824 - val_acc: 0.8656
Epoch 365/500
114/114 [==============================] - 157s 1s/step - loss: 0.1836 - acc: 0.9393 - val_loss: 1.0412 - val_acc: 0.7866
Epoch 366/500
114/114 [==============================] - 157s 1s/step - loss: 0.2052 - acc: 0.9310 - val_loss: 0.7324 - val_acc: 0.8103
Epoch 367/500
114/114 [==============================] - 157s 1s/step - loss: 0.1996 - acc: 0.9319 - val_loss: 0.2833 - val_acc: 0.9328
Epoch 368/500
114/114 [==============================] - 155s 1s/step - loss: 0.1858 - acc: 0.9301 - val_loss: 0.5025 - val_acc: 0.9130
Epoch 369/500
114/114 [==============================] - 155s 

114/114 [==============================] - 154s 1s/step - loss: 0.1761 - acc: 0.9323 - val_loss: 0.3921 - val_acc: 0.9170
Epoch 423/500
114/114 [==============================] - 156s 1s/step - loss: 0.2269 - acc: 0.9389 - val_loss: 0.3814 - val_acc: 0.8972
Epoch 424/500
114/114 [==============================] - 155s 1s/step - loss: 0.1940 - acc: 0.9393 - val_loss: 0.4362 - val_acc: 0.8972
Epoch 425/500
114/114 [==============================] - 157s 1s/step - loss: 0.1642 - acc: 0.9415 - val_loss: 0.3801 - val_acc: 0.8972
Epoch 426/500
114/114 [==============================] - 155s 1s/step - loss: 0.1772 - acc: 0.9415 - val_loss: 0.8030 - val_acc: 0.8458
Epoch 427/500
114/114 [==============================] - 156s 1s/step - loss: 0.2303 - acc: 0.9354 - val_loss: 0.5935 - val_acc: 0.8696
Epoch 428/500
114/114 [==============================] - 156s 1s/step - loss: 0.1963 - acc: 0.9319 - val_loss: 1.0694 - val_acc: 0.8656
Epoch 429/500
114/114 [==============================] - 156s 

114/114 [==============================] - 156s 1s/step - loss: 0.2004 - acc: 0.9336 - val_loss: 0.3715 - val_acc: 0.9249
Epoch 483/500
114/114 [==============================] - 156s 1s/step - loss: 0.2036 - acc: 0.9341 - val_loss: 0.4682 - val_acc: 0.8814
Epoch 484/500
114/114 [==============================] - 156s 1s/step - loss: 0.1905 - acc: 0.9332 - val_loss: 0.3065 - val_acc: 0.9289
Epoch 485/500
114/114 [==============================] - 157s 1s/step - loss: 0.1666 - acc: 0.9499 - val_loss: 0.3781 - val_acc: 0.9170
Epoch 486/500
114/114 [==============================] - 155s 1s/step - loss: 0.2155 - acc: 0.9363 - val_loss: 0.4631 - val_acc: 0.9130
Epoch 487/500
114/114 [==============================] - 156s 1s/step - loss: 0.1724 - acc: 0.9402 - val_loss: 0.4018 - val_acc: 0.9012
Epoch 488/500
114/114 [==============================] - 156s 1s/step - loss: 0.1905 - acc: 0.9341 - val_loss: 0.3351 - val_acc: 0.9209
Epoch 489/500
114/114 [==============================] - 155s 

In [26]:
model.save("ResNet152V2_model.h5")

In [27]:
test_dir = 'aoi/aoi/test_images'
ntest = len(os.listdir("aoi/aoi/test_images/test_images"))
print(ntest)

10142


In [28]:
ntest

10142

In [29]:
test_img=[]
for i in range(ntest):
    path=os.path.join('aoi/aoi/test_images/test_images',os.listdir("aoi/aoi/test_images/test_images")[i])
    test_img.append(preprocess_fn(path))

In [30]:
test_image=np.array(test_img)

In [31]:
test_image.shape

(10142, 224, 224, 3)

In [32]:
test_features=ResNet152V2_conv.predict(test_image[0:2500])

79/79 [==============================] - 371s 5s/step


In [33]:
test_features_5000=ResNet152V2_conv.predict(test_image[2500:5000])

79/79 [==============================] - 368s 5s/step


In [34]:
test_features_7500=ResNet152V2_conv.predict(test_image[5000:7500])

79/79 [==============================] - 371s 5s/step


In [35]:
test_features_10000=ResNet152V2_conv.predict(test_image[7500:10000])

79/79 [==============================] - 374s 5s/step


In [36]:
test_features_10142=ResNet152V2_conv.predict(test_image[10000:10142])

5/5 [==============================] - 21s 4s/step


In [37]:
test_features_input_dim = np.reshape(test_features, (2500, 7*7*2048))

In [38]:
test_features_5000_input_dim = np.reshape(test_features_5000, (2500, 7*7*2048))

In [39]:
test_features_7500_input_dim = np.reshape(test_features_7500, (2500, 7*7*2048))

In [40]:
test_features_10000_input_dim = np.reshape(test_features_10000, (2500, 7*7*2048))

In [41]:
test_features_10142_input_dim = np.reshape(test_features_10142, (142, 7*7*2048))

In [42]:
test_imagepredictions = np.argmax(model.predict(test_features_input_dim),axis=1)

79/79 [==============================] - 7s 63ms/step


In [43]:
test_imagepredictions

array([1, 0, 5, ..., 1, 5, 4], dtype=int64)

In [44]:
test_imagepredictions.shape

(2500,)

In [45]:
test_imagepredictions_5000 = np.argmax(model.predict(test_features_5000_input_dim),axis=1)

79/79 [==============================] - 5s 65ms/step


In [46]:
test_imagepredictions_5000

array([1, 0, 1, ..., 2, 5, 5], dtype=int64)

In [47]:
test_imagepredictions_5000.shape

(2500,)

In [48]:
test_imagepredictions_7500 = np.argmax(model.predict(test_features_7500_input_dim),axis=1)

79/79 [==============================] - 5s 66ms/step


In [49]:
test_imagepredictions_7500

array([1, 5, 1, ..., 5, 1, 1], dtype=int64)

In [50]:
test_imagepredictions_10000 = np.argmax(model.predict(test_features_10000_input_dim),axis=1)

79/79 [==============================] - 5s 60ms/step


In [51]:
test_imagepredictions_10000

array([1, 0, 4, ..., 1, 3, 0], dtype=int64)

In [52]:
test_imagepredictions_10142 = np.argmax(model.predict(test_features_10142_input_dim),axis=1)

5/5 [==============================] - 0s 72ms/step


In [53]:
test_imagepredictions_10142

array([5, 0, 0, 0, 0, 3, 0, 5, 0, 0, 0, 5, 5, 3, 4, 2, 1, 5, 5, 1, 5, 4,
       4, 5, 3, 1, 5, 4, 1, 1, 4, 3, 1, 3, 5, 5, 3, 0, 1, 4, 5, 4, 3, 4,
       0, 1, 5, 5, 3, 4, 4, 4, 0, 2, 1, 1, 5, 1, 3, 3, 5, 1, 1, 1, 4, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 3, 0, 4, 5, 0, 0, 0, 1, 1, 0, 5, 3, 3, 0,
       0, 2, 0, 1, 4, 4, 3, 1, 5, 1, 0, 4, 1, 1, 5, 0, 5, 3, 5, 5, 1, 1,
       0, 4, 0, 0, 1, 4, 1, 0, 5, 5, 0, 1, 0, 5, 3, 0, 5, 4, 3, 2, 5, 0,
       1, 1, 0, 1, 3, 4, 3, 1, 3, 5], dtype=int64)

In [54]:
test_imagepredictions_reshape=test_imagepredictions.reshape((2500,1))
test_imagepredictions_reshape

array([[1],
       [0],
       [5],
       ...,
       [1],
       [5],
       [4]], dtype=int64)

In [55]:
test_imagepredictions_5000_reshape=test_imagepredictions_5000.reshape((2500,1))
test_imagepredictions_5000_reshape

array([[1],
       [0],
       [1],
       ...,
       [2],
       [5],
       [5]], dtype=int64)

In [56]:
test_imagepredictions_7500_reshape=test_imagepredictions_7500.reshape((2500,1))
test_imagepredictions_7500_reshape

array([[1],
       [5],
       [1],
       ...,
       [5],
       [1],
       [1]], dtype=int64)

In [57]:
test_imagepredictions_10000_reshape=test_imagepredictions_10000.reshape((2500,1))
test_imagepredictions_10000_reshape

array([[1],
       [0],
       [4],
       ...,
       [1],
       [3],
       [0]], dtype=int64)

In [58]:
test_imagepredictions_10142_reshape=test_imagepredictions_10142.reshape((142,1))
test_imagepredictions_10142_reshape

array([[5],
       [0],
       [0],
       [0],
       [0],
       [3],
       [0],
       [5],
       [0],
       [0],
       [0],
       [5],
       [5],
       [3],
       [4],
       [2],
       [1],
       [5],
       [5],
       [1],
       [5],
       [4],
       [4],
       [5],
       [3],
       [1],
       [5],
       [4],
       [1],
       [1],
       [4],
       [3],
       [1],
       [3],
       [5],
       [5],
       [3],
       [0],
       [1],
       [4],
       [5],
       [4],
       [3],
       [4],
       [0],
       [1],
       [5],
       [5],
       [3],
       [4],
       [4],
       [4],
       [0],
       [2],
       [1],
       [1],
       [5],
       [1],
       [3],
       [3],
       [5],
       [1],
       [1],
       [1],
       [4],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [3],
       [0],
       [4],
       [5],
       [0],
       [0],
       [0],
       [1],
       [1],
    

In [59]:
aoi_predict = np.concatenate((test_imagepredictions_reshape, 
                              test_imagepredictions_5000_reshape,
                              test_imagepredictions_7500_reshape,
                              test_imagepredictions_10000_reshape,
                              test_imagepredictions_10142_reshape))

In [60]:
aoi_predict

array([[1],
       [0],
       [5],
       ...,
       [1],
       [3],
       [5]], dtype=int64)

In [61]:
aoi_predict.shape

(10142, 1)

In [62]:
test_data = pd.read_csv('aoi/aoi/test.csv',engine='python')
test_data.tail() 

,ID,Label
10137,test_10137.png,4
10138,test_10138.png,1
10139,test_10139.png,1
10140,test_10140.png,3
10141,test_10141.png,1


In [63]:
aoi_predict.reshape((1,10142))

array([[1, 0, 5, ..., 1, 3, 5]], dtype=int64)

In [64]:
aoi_predict.transpose()

array([[1, 0, 5, ..., 1, 3, 5]], dtype=int64)

In [65]:
aoi_predict.transpose()[0:5]

array([[1, 0, 5, ..., 1, 3, 5]], dtype=int64)

In [67]:
import csv

# 開啟輸出的 CSV 檔案
with open('aoi_predict_label.csv', 'w', newline='') as csvfile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvfile)
  # 寫入一列資料
  writer.writerow(aoi_predict)

In [68]:
import csv

# 讀取CSV文件
csv_file = 'aoi_predict_label.csv'
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

# 轉置表格
transposed_data = list(map(list, zip(*data)))

# 寫入轉置後的表格到新的CSV文件
transposed_csv_file = 'aoi_predict_label.csv'
with open(transposed_csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(transposed_data)

In [69]:
import csv

# 讀取CSV文件
csv_file = 'aoi_predict_label.csv'
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

# 去除括號
cleaned_data = []
for row in data:
    cleaned_row = [cell.strip('[]') for cell in row]
    cleaned_data.append(cleaned_row)

# 寫入去除括號後的數據到新的CSV文件
cleaned_csv_file = 'aoi_predict_label.csv'
with open(cleaned_csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(cleaned_data)